# Baseline

In [280]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_validate, StratifiedGroupKFold, KFold

## Загрузка данных

In [281]:
test_df = pd.read_csv("test_df_filled_day_ilnaz.csv")

In [282]:
test_df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster
0,200000,month_4,-0.096224,0.335496,-0.125995,-0.095578,channel_code_12,city_14,city_type_0,NaN,...,0.010952,0.946066,0.407762,-0.153950,0.548895,0.541020,0.031742,0.257278,0.561353,{α}
1,200000,month_5,-0.024255,-0.059806,-0.124295,-0.023381,channel_code_12,city_14,city_type_0,NaN,...,0.006812,0.945281,0.396267,-0.150505,0.549468,0.552131,0.237817,0.264211,0.715199,{α}
2,200000,month_6,0.045988,0.049418,-0.125995,0.047079,channel_code_12,city_14,city_type_0,NaN,...,0.006812,0.945281,0.396267,-0.152800,0.549468,0.541020,0.387566,0.268543,0.836079,{α}
3,200001,month_4,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,NaN,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α}
4,200001,month_5,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_76,city_type_0,NaN,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290115,299998,month_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.028584,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{}
290116,299998,month_6,-0.155350,-0.203711,-0.125995,-0.155980,channel_code_9,city_25,city_type_0,index_city_code_30,...,-0.028584,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,{α}
290117,299999,month_4,-0.145899,-0.173298,-0.125995,-0.145402,channel_code_9,city_6,city_type_0,index_city_code_34,...,-0.028584,0.944497,0.384773,-0.097640,0.550901,0.574354,-0.101369,0.252657,0.440474,{α}
290118,299999,month_5,-0.136402,-0.163874,-0.121475,-0.135876,channel_code_9,city_6,city_type_0,index_city_code_34,...,-0.028584,0.944497,0.384773,-0.047885,0.555057,0.696576,-0.034860,0.253523,0.473441,{α}


In [283]:
city_counts = test_df['city'].value_counts().to_dict()
#city_counts
test_df["city_other"] = test_df["city"].apply(lambda x: x  if city_counts.get(x, 0) > 15  else "other")
test_df.drop("city", axis=1, inplace=True)
test_df.city_other.value_counts()

city_to_index_mapping = test_df.dropna(subset=['city_other', 'index_city_code']).drop_duplicates('index_city_code').set_index('index_city_code')['city_other'].to_dict()


for idx, row in test_df.iterrows():
    if pd.isna(row['city_other']):
        index_city_code = row['index_city_code']

        if index_city_code in city_to_index_mapping:
            test_df.at[idx, 'city_other'] = city_to_index_mapping[index_city_code]

        elif not pd.isna(index_city_code):
            test_df.at[idx, 'city_other'] = index_city_code

test_df["city"] = test_df["city_other"]
test_df.drop("city_other", axis=1, inplace=True)

In [284]:


# city_to_index_mapping = test_df.dropna(subset=['city', 'index_city_code']).drop_duplicates('index_city_code').set_index('index_city_code')['city'].to_dict()
#
#
# for idx, row in test_df.iterrows():
#     if pd.isna(row['city']):
#         index_city_code = row['index_city_code']
#
#         if index_city_code in city_to_index_mapping:
#             test_df.at[idx, 'city'] = city_to_index_mapping[index_city_code]
#
#         elif not pd.isna(index_city_code):
#             test_df.at[idx, 'city'] = index_city_code


In [285]:
test_df.head(30)

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city_type,index_city_code,ogrn_days_end_month,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,city
0,200000,month_4,-0.096224,0.335496,-0.125995,-0.095578,channel_code_12,city_type_0,NaN,-1.533705,...,0.946066,0.407762,-0.153950,0.548895,0.541020,0.031742,0.257278,0.561353,{α},city_14
1,200000,month_5,-0.024255,-0.059806,-0.124295,-0.023381,channel_code_12,city_type_0,NaN,-1.533705,...,0.945281,0.396267,-0.150505,0.549468,0.552131,0.237817,0.264211,0.715199,{α},city_14
2,200000,month_6,0.045988,0.049418,-0.125995,0.047079,channel_code_12,city_type_0,NaN,-1.533705,...,0.945281,0.396267,-0.152800,0.549468,0.541020,0.387566,0.268543,0.836079,{α},city_14
3,200001,month_4,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_type_0,NaN,0.092087,...,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α},city_76
4,200001,month_5,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_type_0,NaN,0.092087,...,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α},city_76
5,200001,month_6,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_type_0,NaN,0.092087,...,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α},city_76
6,200002,month_4,-0.096506,0.185905,-0.125995,-0.095856,channel_code_12,city_type_0,index_city_code_78,-1.069193,...,0.950774,0.545693,0.715525,0.554913,0.718798,0.445811,0.254968,0.495419,{other},city_14
7,200002,month_5,0.572242,1.502779,-0.125995,0.574963,channel_code_12,city_type_0,index_city_code_78,-1.069193,...,0.949989,0.522704,2.442243,0.563940,0.841020,3.313686,0.256701,0.550364,{other},city_14
8,200002,month_6,-0.048015,0.448252,-0.125995,-0.047215,channel_code_12,city_type_0,index_city_code_78,-1.069193,...,0.946850,0.453739,2.614870,0.565087,0.818798,4.449125,0.258723,0.627287,{other},city_14
9,200003,month_4,-0.156623,-0.204813,-0.125665,-0.156159,channel_code_7,city_type_0,NaN,-0.256297,...,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α},city_31


In [286]:

test_df['month_number'] = test_df.groupby('id').cumcount() + 1


test_df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city_type,index_city_code,ogrn_days_end_month,...,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,city,month_number
0,200000,month_4,-0.096224,0.335496,-0.125995,-0.095578,channel_code_12,city_type_0,NaN,-1.533705,...,0.407762,-0.153950,0.548895,0.541020,0.031742,0.257278,0.561353,{α},city_14,1
1,200000,month_5,-0.024255,-0.059806,-0.124295,-0.023381,channel_code_12,city_type_0,NaN,-1.533705,...,0.396267,-0.150505,0.549468,0.552131,0.237817,0.264211,0.715199,{α},city_14,2
2,200000,month_6,0.045988,0.049418,-0.125995,0.047079,channel_code_12,city_type_0,NaN,-1.533705,...,0.396267,-0.152800,0.549468,0.541020,0.387566,0.268543,0.836079,{α},city_14,3
3,200001,month_4,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_type_0,NaN,0.092087,...,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α},city_76,1
4,200001,month_5,-0.156722,-0.204920,-0.125856,-0.156258,channel_code_9,city_type_0,NaN,0.092087,...,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{α},city_76,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290115,299998,month_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},other,1
290116,299998,month_6,-0.155350,-0.203711,-0.125995,-0.155980,channel_code_9,city_type_0,index_city_code_30,0.672727,...,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,{α},city_25,2
290117,299999,month_4,-0.145899,-0.173298,-0.125995,-0.145402,channel_code_9,city_type_0,index_city_code_34,-1.185321,...,0.384773,-0.097640,0.550901,0.574354,-0.101369,0.252657,0.440474,{α},city_6,1
290118,299999,month_5,-0.136402,-0.163874,-0.121475,-0.135876,channel_code_9,city_type_0,index_city_code_34,-1.185321,...,0.384773,-0.047885,0.555057,0.696576,-0.034860,0.253523,0.473441,{α},city_6,2


In [287]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290120 entries, 0 to 290119
Data columns (total 93 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       290120 non-null  int64  
 1   date                     290120 non-null  object 
 2   balance_amt_avg          260637 non-null  float64
 3   balance_amt_max          260637 non-null  float64
 4   balance_amt_min          260637 non-null  float64
 5   balance_amt_day_avg      260637 non-null  float64
 6   channel_code             270092 non-null  object 
 7   city_type                229128 non-null  object 
 8   index_city_code          133659 non-null  object 
 9   ogrn_days_end_month      275842 non-null  float64
 10  ogrn_days_end_quarter    275842 non-null  float64
 11  ogrn_month               275842 non-null  object 
 12  ogrn_year                275842 non-null  object 
 13  ft_registration_date     275282 non-null  float64
 14  max_

In [288]:
pivot_columns = [col for col in test_df.columns if col not in ['id', 'date', 'month_number']]

df_pivoted = test_df.pivot_table(index='id', columns='month_number', values=pivot_columns, aggfunc='first')

# Теперь, когда мы сформировали сводную таблицу, преобразуем MultiIndex столбцы в единый уровень
df_pivoted.columns = [f'{col[0]}_{col[1]}month' for col in df_pivoted.columns]

# Переименовываем индекс обратно в столбец 'id', если он стал индексом
df_pivoted.reset_index(inplace=True)

df_pivoted.head()



,id,balance_amt_avg_1month,balance_amt_avg_2month,balance_amt_avg_3month,balance_amt_day_avg_1month,balance_amt_day_avg_2month,balance_amt_day_avg_3month,balance_amt_max_1month,balance_amt_max_2month,balance_amt_max_3month,...,sum_deb_h_oper_3m_3month,sum_of_paym_1y_1month,sum_of_paym_1y_2month,sum_of_paym_1y_3month,sum_of_paym_2m_1month,sum_of_paym_2m_2month,sum_of_paym_2m_3month,sum_of_paym_6m_1month,sum_of_paym_6m_2month,sum_of_paym_6m_3month
0,200000,-0.096224,-0.024255,0.045988,-0.095578,-0.023381,0.047079,0.335496,-0.059806,0.049418,...,-0.152800,0.676573,0.688449,0.671862,0.416833,0.433195,0.223961,0.332409,0.284317,0.285376
1,200001,-0.156722,-0.156722,-0.156722,-0.156258,-0.156258,-0.156258,-0.204920,-0.204920,-0.204920,...,-0.165588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200002,-0.096506,0.572242,-0.048015,-0.095856,0.574963,-0.047215,0.185905,1.502779,0.448252,...,2.614870,0.365591,0.970531,1.211625,1.303965,3.870911,4.142519,0.550373,1.620822,1.969559
3,200003,-0.156623,-0.156579,-0.156579,-0.156159,-0.156115,-0.156115,-0.204813,-0.204813,-0.204813,...,-0.165588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200004,NaN,-0.153379,NaN,NaN,-0.154155,NaN,NaN,-0.201932,NaN,...,NaN,NaN,-0.296092,NaN,NaN,-0.273794,NaN,NaN,-0.294571,NaN


In [289]:
df_pivoted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 271 entries, id to sum_of_paym_6m_3month
dtypes: float64(243), int64(1), object(27)
memory usage: 206.8+ MB


In [290]:
X = df_pivoted.drop([
"cnt_a_oper_1m_1month", "cnt_a_oper_1m_2month", "cnt_a_oper_1m_3month",
"cnt_b_oper_1m_1month", "cnt_b_oper_1m_2month", "cnt_b_oper_1m_3month",
"cnt_c_oper_1m_1month", "cnt_c_oper_1m_2month", "cnt_c_oper_1m_3month",
"cnt_deb_d_oper_1m_1month", "cnt_deb_d_oper_1m_2month", "cnt_deb_d_oper_1m_3month",
"cnt_cred_d_oper_1m_1month", "cnt_cred_d_oper_1m_2month", "cnt_cred_d_oper_1m_3month",
"cnt_deb_e_oper_1m_1month", "cnt_deb_e_oper_1m_2month", "cnt_deb_e_oper_1m_3month",
"cnt_cred_e_oper_1m_1month", "cnt_cred_e_oper_1m_2month", "cnt_cred_e_oper_1m_3month",
"cnt_deb_f_oper_1m_1month", "cnt_deb_f_oper_1m_2month", "cnt_deb_f_oper_1m_3month",
"cnt_cred_f_oper_1m_1month", "cnt_cred_f_oper_1m_2month", "cnt_cred_f_oper_1m_3month",
"cnt_deb_g_oper_1m_1month", "cnt_deb_g_oper_1m_2month", "cnt_deb_g_oper_1m_3month",
"cnt_cred_g_oper_1m_1month", "cnt_cred_g_oper_1m_2month", "cnt_cred_g_oper_1m_3month",
"cnt_deb_h_oper_1m_1month", "cnt_deb_h_oper_1m_2month", "cnt_deb_h_oper_1m_3month",
"cnt_cred_h_oper_1m_1month", "cnt_cred_h_oper_1m_2month", "cnt_cred_h_oper_1m_3month",
"cnt_a_oper_3m_1month", "cnt_a_oper_3m_2month", "cnt_a_oper_3m_3month",
"cnt_b_oper_3m_1month", "cnt_b_oper_3m_2month", "cnt_b_oper_3m_3month",
"cnt_c_oper_3m_1month", "cnt_c_oper_3m_2month", "cnt_c_oper_3m_3month",
"cnt_deb_d_oper_3m_1month", "cnt_deb_d_oper_3m_2month", "cnt_deb_d_oper_3m_3month",
"cnt_cred_d_oper_3m_1month", "cnt_cred_d_oper_3m_2month", "cnt_cred_d_oper_3m_3month",
"cnt_deb_e_oper_3m_1month", "cnt_deb_e_oper_3m_2month", "cnt_deb_e_oper_3m_3month",
"cnt_cred_e_oper_3m_1month", "cnt_cred_e_oper_3m_2month", "cnt_cred_e_oper_3m_3month",
"cnt_deb_f_oper_3m_1month", "cnt_deb_f_oper_3m_2month", "cnt_deb_f_oper_3m_3month",
"cnt_cred_f_oper_3m_1month", "cnt_cred_f_oper_3m_2month", "cnt_cred_f_oper_3m_3month",
"cnt_deb_g_oper_3m_1month", "cnt_deb_g_oper_3m_2month", "cnt_deb_g_oper_3m_3month",
"cnt_cred_g_oper_3m_1month", "cnt_cred_g_oper_3m_2month", "cnt_cred_g_oper_3m_3month",
"cnt_deb_h_oper_3m_1month", "cnt_deb_h_oper_3m_2month", "cnt_deb_h_oper_3m_3month",
"cnt_cred_h_oper_3m_1month", "cnt_cred_h_oper_3m_2month", "cnt_cred_h_oper_3m_3month",

"city_type_1month",
"city_type_2month",
"city_type_3month",
"index_city_code_1month",
"index_city_code_2month",
"index_city_code_3month",

    "start_cluster_3month","id"], axis=1)


In [291]:
X

,balance_amt_avg_1month,balance_amt_avg_2month,balance_amt_avg_3month,balance_amt_day_avg_1month,balance_amt_day_avg_2month,balance_amt_day_avg_3month,balance_amt_max_1month,balance_amt_max_2month,balance_amt_max_3month,balance_amt_min_1month,...,sum_deb_h_oper_3m_3month,sum_of_paym_1y_1month,sum_of_paym_1y_2month,sum_of_paym_1y_3month,sum_of_paym_2m_1month,sum_of_paym_2m_2month,sum_of_paym_2m_3month,sum_of_paym_6m_1month,sum_of_paym_6m_2month,sum_of_paym_6m_3month
0,-0.096224,-0.024255,0.045988,-0.095578,-0.023381,0.047079,0.335496,-0.059806,0.049418,-0.125995,...,-0.152800,0.676573,0.688449,0.671862,0.416833,0.433195,0.223961,0.332409,0.284317,0.285376
1,-0.156722,-0.156722,-0.156722,-0.156258,-0.156258,-0.156258,-0.204920,-0.204920,-0.204920,-0.125856,...,-0.165588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.096506,0.572242,-0.048015,-0.095856,0.574963,-0.047215,0.185905,1.502779,0.448252,-0.125995,...,2.614870,0.365591,0.970531,1.211625,1.303965,3.870911,4.142519,0.550373,1.620822,1.969559
3,-0.156623,-0.156579,-0.156579,-0.156159,-0.156115,-0.156115,-0.204813,-0.204813,-0.204813,-0.125665,...,-0.165588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,-0.153379,NaN,NaN,-0.154155,NaN,NaN,-0.201932,NaN,NaN,...,NaN,NaN,-0.296092,NaN,NaN,-0.273794,NaN,NaN,-0.294571,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-0.153707,-0.153707,-0.153707,-0.153247,-0.153234,-0.153234,-0.202806,-0.202806,-0.202806,-0.125498,...,-0.165588,-0.296115,-0.296115,-0.296115,-0.273969,-0.273969,-0.273969,-0.294633,-0.294633,-0.294633
99996,-0.156741,-0.156064,-0.154929,-0.156277,-0.155598,-0.154459,-0.204912,-0.203645,-0.197878,-0.125995,...,-0.155776,-0.289371,-0.289810,-0.287516,-0.273496,-0.272045,-0.259945,-0.284472,-0.283784,-0.284207
99997,-0.125643,-0.121206,-0.105294,-0.125084,-0.120632,-0.104671,-0.098937,-0.148644,-0.141429,-0.124644,...,0.087901,-0.137811,-0.111877,-0.084739,-0.070691,-0.041622,-0.008728,-0.098975,-0.082558,-0.068838
99998,NaN,-0.155350,NaN,NaN,-0.155980,NaN,NaN,-0.203711,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Обучение модели

В качестве базовой модели возьмем LGBM обучим на всех признаках

In [292]:
from sklearn.model_selection import StratifiedKFold, LeavePOut, RepeatedStratifiedKFold
from catboost import  CatBoostClassifier


cat_model = CatBoostClassifier()
model = cat_model.load_model(f'my_meth6_d6_slow\model_12.cbm')


In [293]:
cluster_weights = pd.read_excel("cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()
# weights_dict
cat_features = [col for col in X.columns if X[col].dtype.name == 'category' or X[col].dtype.name == 'object']
for col in cat_features:
    X[col] = X[col].astype('category')
    #test_df[col] = test_df[col].astype('category')


cat_features = [col for col in X.columns if X[col].dtype.name == 'category' or X[col].dtype.name == 'object']
for col in cat_features:
    X[col] = X[col].cat.add_categories('Missing').fillna('Missing')
cat_features

cat_features = [col for col in X.columns if X[col].dtype.name == 'category' or X[col].dtype.name == 'object']
cat_features

['channel_code_1month',
 'channel_code_2month',
 'channel_code_3month',
 'city_1month',
 'city_2month',
 'city_3month',
 'ogrn_month_1month',
 'ogrn_month_2month',
 'ogrn_month_3month',
 'ogrn_year_1month',
 'ogrn_year_2month',
 'ogrn_year_3month',
 'okved_1month',
 'okved_2month',
 'okved_3month',
 'segment_1month',
 'segment_2month',
 'segment_3month',
 'start_cluster_1month',
 'start_cluster_2month']

In [294]:
ppredictions = model.predict_proba(X)

In [295]:
# ppredictions = ppredictions.T

In [296]:
sample_submission_df = pd.read_csv("sample_submission.csv")

In [297]:
ppredictions

array([[1.84399202e-02, 3.13123204e-02, 1.32034666e-02, ...,
        1.52018979e-03, 8.60982642e-01, 7.25861306e-06],
       [7.40368390e-03, 5.62184621e-01, 3.84422335e-04, ...,
        2.67351779e-04, 4.16045290e-01, 2.04394888e-05],
       [5.78845714e-01, 3.33200500e-03, 4.13205516e-03, ...,
        7.13523304e-02, 1.23364152e-01, 1.12933191e-05],
       ...,
       [4.21104002e-02, 2.90238509e-02, 2.79025863e-02, ...,
        2.30264008e-02, 7.41107300e-01, 5.64583395e-06],
       [8.34621930e-02, 2.97295078e-01, 2.97215295e-02, ...,
        2.40464690e-03, 4.23651071e-01, 8.02864605e-04],
       [3.48011365e-02, 1.03159330e-01, 1.34235415e-02, ...,
        6.36378737e-03, 7.33523988e-01, 7.67489453e-05]])

In [298]:
test_pred_proba_df = pd.DataFrame(ppredictions, columns=model.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]
test_pred_proba_df.shape


(100000, 17)

In [299]:
test_pred_proba_df.head(3)

,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,0.018440,0.031312,0.013203,0.023858,0.009223,0.000369,0.005917,0.000390,0.013434,0.005368,0.014335,0.000483,0.001151,0.000007,0.001520,0.860983,0.000007
1,0.007404,0.562185,0.000384,0.002696,0.000564,0.000141,0.000294,0.000016,0.001546,0.005867,0.001293,0.000170,0.001105,0.000002,0.000267,0.416045,0.000020
2,0.578846,0.003332,0.004132,0.108373,0.012536,0.001766,0.007453,0.010951,0.048117,0.010137,0.009197,0.009122,0.001301,0.000008,0.071352,0.123364,0.000011


In [300]:

sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("submissions/meth6_d6/Tima_meth6_12_5_co15.csv", index=False)